<a href="https://colab.research.google.com/github/rahulmk8055/AI-vs-Human-Text-Classification/blob/main/LLM_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install torch transformers datasets scikit-learn pandas numpy
! pip install accelerate -U
! pip install transformers[torch]

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [4]:
dataset = load_dataset("artem9k/ai-text-detection-pile")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [15]:
df = dataset['train'].to_pandas()

# Balance the dataset
human_samples = df[df['source'] == 'human'].sample(n=len(df[df['source'] == 'ai']), random_state=42)
ai_samples = df[df['source'] == 'ai']
balanced_df = pd.concat([human_samples, ai_samples]).reset_index(drop=True)

# Use only 3% of the balanced dataset
sample_size = int(len(balanced_df) * 0.01)
final_df = balanced_df.sample(n=sample_size, random_state=42)

# Split into features and labels
X = final_df['text']
y = (final_df['source'] == 'ai').astype(int)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")
print(f"Percentage of AI-generated texts in training set: {y_train.mean()*100:.2f}%")

Training set size: 5829
Testing set size: 1458
Percentage of AI-generated texts in training set: 49.61%


In [16]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True, max_length=512)

train_encodings = tokenize_function(X_train.tolist())
test_encodings = tokenize_function(X_test.tolist())


In [17]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, y_train.tolist())
test_dataset = TextDataset(test_encodings, y_test.tolist())


In [18]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = np.mean(predictions == labels)
    return {"accuracy": accuracy}


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.226400,0.181882,0.929355
2,0.248300,0.300555,0.923182
3,0.004300,0.329946,0.937586


TrainOutput(global_step=1095, training_loss=0.19128223322246854, metrics={'train_runtime': 527.0409, 'train_samples_per_second': 33.18, 'train_steps_per_second': 2.078, 'total_flos': 2316457400297472.0, 'train_loss': 0.19128223322246854, 'epoch': 3.0})

In [23]:
results = trainer.evaluate()
print(f"Test accuracy: {results['eval_accuracy']:.4f}")


Test accuracy: 0.9376


In [30]:
def predict_source(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to('cuda') # Move input tensors to GPU
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "AI-generated" if prediction == 1 else "Human-written"

# Example usage
sample_text = " The type of safety devices you should install in your home depend on your specific needs, but some of the most common safety devices include smoke and carbon monoxide detectors, security systems, fire extinguishers, and window locks. Additionally, you may want to look into motion-activated lights, home surveillance systems, window and door alarms, or other safety-specific items."
print(f"The text is likely {predict_source(sample_text)}")


The text is likely AI-generated
